In [1]:
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

In [2]:
import astro_time
import sgp4
import sensor

import tle_fitter

In [3]:
import public_astrostandards as PA
# PA.get_versions()

In [15]:
# these are raw UDL obs
def prepObs( o_df ):
    o_df['obTime_dt'] = pd.to_datetime( o_df['obTime'] )
    o_df.sort_values(by='obTime_dt', inplace=True )
    t_df = astro_time.convert_times( obs['obTime_dt'], PA )
    o_df = pd.concat( (t_df.reset_index(drop=True),obs.reset_index(drop=True) ), axis=1 )
    return o_df 
obs    = pd.read_json('./19548.json.gz').sort_values(by='obTime').reset_index(drop=True)
obs_df = prepObs( obs )    

In [7]:
def rotateTEMEObs( O , harness ):
    newRA  = (harness.ctypes.c_double)()
    newDec = (harness.ctypes.c_double)()
    PA.AstroFuncDll.RotRADec_EqnxToDate( 106,
                                        2,
                                        O['ds50_utc'],
                                        O['ra'],
                                        O['declination'],
                                        newRA,
                                        newDec )
    return ( np.float64(newRA),np.float64(newDec) )

def rotateTEMEdf( df, harness ):
    tv = df.apply( lambda X : rotateTEMEObs( X, harness ) , axis=1 )
    df['teme_ra'] = [ X[0] for X in tv ]
    df['teme_dec'] = [ X[1] for X in tv ]
    x = np.cos( np.radians(df['teme_dec']) ) * np.cos( np.radians( df['teme_ra'] ) )
    y = np.cos( np.radians(df['teme_dec']) ) * np.sin( np.radians( df['teme_ra'] ) )
    z = np.sin( np.radians(df['teme_dec'] ) )
    lv = np.hstack( ( x.values[:,np.newaxis], y.values[:,np.newaxis], z.values[:,np.newaxis] )  )
    df['teme_lv'] = lv.tolist()
    return df

obs_df = rotateTEMEdf( obs_df, PA )[['ra','teme_ra']]

In [9]:
L1 = '1 19548U 88091B   25281.05493527 -.00000297  00000+0  00000+0 0  9993'
L2 = '2 19548  12.7961 342.6596 0038175 340.8908  24.1736  1.00278194122860'

In [ ]:
# -----------------------------------------------------------------------------------------------------
def optFunction( X, EH, return_scalar=True ):
    PA      = EH.PA
    XS_TLE  = PA.Cstr('',512)
    # take the function parameters (X) and overwrite the "new_tle" values based on FIELDS 
    for k,v in zip(EH.FIELDS,X) :  EH.new_tle[ k ] = v
    # --------------------- clear state
    PA.TleDll.TleRemoveAllSats()
    PA.Sgp4PropDll.Sgp4RemoveAllSats()
    # --------------------- init our test TLE from the modified data
    tleid = PA.TleDll.TleAddSatFrArray( EH.new_tle.data, XS_TLE )
    if tleid <= 0: return np.inf
    if PA.Sgp4PropDll.Sgp4InitSat( tleid ) != 0: return np.inf
    # --------------------- generate our test ephemeris
    test_eph = sgp4.propTLEToDS50s( tleid, EH.truth_date, PA )
    # use numpy to return the distance between our hypothesis and truth
    resids = test_eph[:,1:4] - EH.truth_eph
    rms    = np.sqrt( np.sum( np.linalg.norm( resids, axis=1 ) ) / resids.shape[0] )
    print( 'RMS : {:08.3f}                    '.format(rms) , end='\r')
    if return_scalar:
        return rms
        # return np.sum( np.linalg.norm( resids[:,:3], axis=1 ) ) 
    else:
        np.linalg.norm( resids[:,:3], axis=1 ) 

In [23]:
# -----------------------------------------------------------------------------------------------------
class eo_fitter( tle_fitter.tle_fitter ):
    def __init__( self, PA ):
        super().__init__( PA )
        self.line1 = None
        self.line2 = None

    def _init_fields( self ):
        # epoch at the last ob (assume sorted)
        self.epoch_idx = -1
        epoch_sv        = self.obs_df.iloc[ self.epoch_idx ]
        osc_data        = tle_fitter.sv_to_osc( epoch_sv , self.PA )
        mean_data       = tle_fitter.osc_to_mean( osc_data, self.PA )
        # update our "fit" TLE with the new osculating data
        self.new_tle    = tle_fitter.insert_kep_to_TLE( self.new_tle, mean_data, self.PA )
        # if this is a type-0, we need Kozai mean motion   
        if self.new_tle['XA_TLE_EPHTYPE'] == 0:
            self.new_tle['XA_TLE_MNMOTN'] = self.PA.AstroFuncDll.BrouwerToKozai( 
                    self.new_tle['XA_TLE_ECCEN'], 
                    self.new_tle['XA_TLE_INCLI'],
                    self.new_tle['XA_TLE_MNMOTN'] )
        self.new_tle['XA_TLE_EPOCH'] = epoch_sv['ds50_utc']
        return self

    def set_data( self, L1 : str, L2 : str, obs : list[ dict ] ):
        ''' 
        take an initial TLE as a guess

        take a list of JSON formatted obs (directly from UDL)
        '''
        self.obs_df     = prepObs( obs )
        self.obs_df     = rotateTEMEdf( self.obs_df, self.PA )
        self._look_vecs = np.vstack( self.obs_df['teme_lv'] )
        self._ds50_utc  = self.obs_df['ds50_utc']
                                            

    
A = eo_fitter( PA ).set_data( L1, L2, obs_df )